In [1]:
import nashpy as nash
import numpy as np
from typing import Tuple

NUM_EVENTS = 3
NUM_SIGNALS = 3

def generate_matrices_one_to_one(prob_distribution: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """ prob_distribution: 1 x NUM_EVENTS
        signal_costs: 1 x NUM_SIGNALS
        sender_matrix: NUM_EVENTS x NUM_SIGNALS
        receiver_matrix: NUM_SIGNALS x NUM_EVENTS
    """
    base_sender_matrix = np.zeros((NUM_EVENTS, NUM_SIGNALS))
    np.fill_diagonal(base_sender_matrix, 1)
    sender_matrix =  (base_sender_matrix.T * prob_distribution).T
    
    receiver_matrix = np.zeros((NUM_SIGNALS, NUM_EVENTS))
    np.fill_diagonal(receiver_matrix, 1)
    
    return sender_matrix, receiver_matrix

def payoff(sender_matrix: np.ndarray, receiver_matrix: np.ndarray, signal_costs: np.ndarray) -> float:
    """ sender_matrix: NUM_EVENTS x NUM_SIGNALS
        receiver_matrix: NUM_SIGNALS x NUM_EVENTS
        signal_costs: 1 x NUM_SIGNALS
    """
    return np.trace(np.matmul(sender_matrix, receiver_matrix/signal_costs))

probs = [0.8, 0.1, 0.1]
costs = [5, 1, 1]
s, r = generate_matrices_one_to_one(probs)
payoff(s, r, costs)
    

0.36

In [77]:
# 2d experiment
from scipy.optimize import minimize, differential_evolution, NonlinearConstraint

# s_ij - the probability of the sender mapping event i to signal j
# e_ji - the probability of the receiver mapping signal j to event i

# vector: x = [s_11, s_12, s_21, s_22, e_11, e_21, e_12, e_22]
PROB_VECTOR = [0.8, 0.5]
COST_VECTOR = [5, 1]
# "global" for diff_evolution, "local" to find one min
OPT_TYPE = "global"

# note this library works on minimization problems, so I write the negative of the true objective
def objective(x: np.ndarray) -> float:
    return -1 * (PROB_VECTOR[0] * (1/COST_VECTOR[0]) * (x[0] * x[4]) + PROB_VECTOR[0] * (1/COST_VECTOR[1]) * (x[1] * x[6]) + \
PROB_VECTOR[1] * (1/COST_VECTOR[0]) * (x[2] * x[5]) + PROB_VECTOR[1] * (1/COST_VECTOR[1]) * (x[3] * x[7]))

# s_11 + s_12 = 1
def constraint_1(x: np.ndarray) -> float:
    return x[0] + x[1] - 1

# s_21 + s_22 = 1
def constraint_2(x: np.ndarray) -> float:
    return x[2] + x[3] - 1

# e_11 + e_21 = 1
def constraint_3(x: np.ndarray) -> float:
    return x[4] + x[5] - 1

# e_12 + e_22
def constraint_4(x: np.ndarray) -> float:
    return x[6] + x[7] - 1

if OPT_TYPE == "local":
    all_constraints = ([{"type": "eq", "fun": constraint_1},{"type": "eq", "fun": constraint_2}, {"type": "eq", "fun": constraint_3},
                       {"type": "eq", "fun": constraint_4}])
    x0 = np.random.uniform(low=0, high=1, size=(8,))
    b = (0, 1)
    bounds = tuple([b] * 8)
    solution = minimize(objective,x0,method='SLSQP',\
                        bounds=bounds,constraints=all_constraints)
    x = solution.x
    print(x)

    print('Final SSE Objective: ' + str(objective(x)))

elif OPT_TYPE == "global":
    constraints = ( NonlinearConstraint(constraint_1, 0, 0),
                   NonlinearConstraint(constraint_2, 0, 0),
                   NonlinearConstraint(constraint_3, 0, 0),
                   NonlinearConstraint(constraint_4, 0, 0),
                  )
    
    solution = differential_evolution(objective,bounds=bounds,workers=-1,maxiter=10000,constraints=constraints)
    print(solution)

           constr: [array([0.]), array([0.]), array([0.]), array([0.])]
 constr_violation: 0.0
              fun: -0.8986265304677151
            maxcv: 0.0
          message: 'Optimization terminated successfully.'
             nfev: 10237
              nit: 7327
          success: True
                x: array([3.35615246e-04, 9.99664385e-01, 9.97441430e-01, 2.55856987e-03,
       5.13045565e-03, 9.94869544e-01, 9.99577103e-01, 4.22896868e-04])


In [35]:
sender = np.array([[0.5, 0], [0, 0.5]])
receiver = np.array([[1, 0], [0, 1]])
print(payoff(sender, receiver, COST_VECTOR))

0.55
